In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rdflib import Literal, RDF, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph
import rdflib.namespace
from owlready2 import *
from owlready2 import get_ontology
import networkx as nx
import networkx.algorithms.community as nx_comm

In [ ]:
onto = get_ontology("http://example.org/medsur.owl")

In [ ]:
class Patients(Thing):
    namespace = onto

class AgeGroup(Thing):
    namespace = onto
 
class hasAgeGroup(ObjectProperty):                 
    domain = [Patients]
    range = [AgeGroup]
    namespace = onto
  
class WeightGroup(Thing):
    namespace = onto
    
class hasWeightGroup(ObjectProperty):   
    domain = [Patients]
    range = [WeightGroup]
    namespace = onto
  
class SOC(Thing):  #System Organ Class
    namespace = onto
      
class hasSOC(ObjectProperty):
    domain = [Patients]
    range = [SOC]
    namespace = onto
    
class HLGT(SOC):  #High Level Group Term
    namespace = onto
    
class hasHLGT(ObjectProperty):
    domain = [Patients]	
    range = [HLGT]
    namespace = onto

class HLT(HLGT):  #High Level Term
    namespace = onto
    range = [HLGT]
    namespace = onto

class hasHLT(ObjectProperty):
    domain = [Patients]
    range = [HLT]
    namespace = onto
    
class PT(HLT):  #Preferred Term 
    namespace = onto
    
class hasPT(ObjectProperty):
    domain = [Patients]
    range = [PT]
    namespace = onto

class LLT(PT):  #Low Level Term
    namespace = onto
    
class hasLLT(ObjectProperty):  
    domain = [Patients]
    range = [LLT]
    namespace = onto

class Outcome(Thing):
    namespace = onto
    
class hasOutcome(ObjectProperty):
    domain = [Patients]
    range = [Outcome]
    namespace = onto
    
class Gender(Thing):
    namespace = onto

class hasGender(ObjectProperty):
    domain = [Patients]
    range = [Gender]
    namespace = onto

class Drug(Thing):
    namespace = onto
    
class IsGivenDrug(ObjectProperty):
    domain = [Patients]
    range = [Drug]
    namespace = onto
    
class IsOfDosis(ObjectProperty):
    domain = [Drug]
    namespace = onto
    
class IsOfType(ObjectProperty):
    domain = [Drug]
    namespace = onto

class SideEffects(PT):   # subclass of PT
    namespace = onto
    
class hasSideEffect(ObjectProperty):
    domain = [Drug]
    range = [SideEffects]
    namespace = onto
    
class hasFrequency(ObjectProperty):                 
    domain = [SideEffects]
    namespace = onto 
    
class suffersSideEffects(ObjectProperty):
    domain = [Patients]
    range = [SideEffects]
    namespace = onto

In [ ]:
# save the ontology
onto.save(file = "medsur.rdf", format = "rdfxml") 
g = rdflib.Graph()
g.parse("medsur.rdf", format="xml")

# Loop through each triple in the graph (subj, pred, obj)
for subj, pred, obj in g:
    
    # Check if there is at least one triple in the Graph
    if (subj, pred, obj) not in g:
       raise Exception("It better be!")

# Print the number of "triples" in the Graph
print(f"Graph g has {len(g)} statements.")

In [ ]:
# print all the triples in the graph 
for s, p, o in g:
    print(s, p, o)

In [ ]:
data = pd.read_excel('opioid_datamerged.xlsx')

# add index column
data = data.reset_index()

data_sideeffects = pd.read_excel('sider_output.xlsx')

In [ ]:
# add RDF triples to the ontology
EX = rdflib.Namespace("http://example.org/medsur.rdf#")

for index, row in data.iterrows():

    patient = URIRef(f"http://www.medsur.org/patient_{index}")
    g.add((patient, RDF.type, EX.Patients))

    if row["weight_group"] != np.nan:
        weight_group = URIRef(f"http://www.medsur.org/weight/{row['weight_group']}")
        g.add((weight_group, RDF.type, EX.WeightGroup))
        g.add((patient, EX.hasWeightGroup, weight_group))

    if float(row["age_year"]) >= 65:
        agegroup = URIRef("http://www.medsur.org/age/65_above")
    elif float(row["age_year"]) >= 45:
        agegroup = URIRef("http://www.medsur.org/age/45_64") 
    elif float(row["age_year"]) >= 25:
        agegroup = URIRef("http://www.medsur.org/age/25_44")
    elif float(row["age_year"]) >= 18:
        agegroup = URIRef("http://www.medsur.org/age/18_24")
    
    if agegroup:
        g.add((agegroup, RDF.type, EX.AgeGroup))
        g.add((patient, EX.hasAgeGroup, agegroup))

    if row["sex"] == "male" or row["sex"] == "female":
        gender = URIRef(f"http://www.medsur.org/gender/{row['sex']}")
        g.add((gender, RDF.type, EX.Gender))
        g.add((patient, EX.hasGender, gender))

    if row["Outcome"] != "Unknown" and row["Outcome"] != np.nan: 
        outcome = URIRef(f"http://www.medsur.org/outcome/{row['Outcome']}")
        g.add((outcome, RDF.type, EX.Outcome))
        g.add((patient, EX.hasOutcome, outcome))

    if row["SOCCode"] != np.nan:
        soc = URIRef(f"http://www.medsur.org/soc/{row['SOCCode']}")
        g.add((soc, RDF.type, EX.SOC))
        g.add((patient, EX.hasSOC, soc))
        
    if row["HLTGCode"] != np.nan:
        hlgt = URIRef(f"http://www.medsur.org/hlgt/{row['HLTGCode']}")
        g.add((hlgt, RDF.type, EX.HLGT))
        g.add((patient, EX.hasHLGT, hlgt))
        
    if row["HLTCode"] != np.nan:
        hlt = URIRef(f"http://www.medsur.org/hlt/{row['HLTCode']}")
        g.add((hlt, RDF.type, EX.HLT))
        g.add((patient, EX.hasHLT, hlt))
    
    if row["PTCode"] != np.nan:
        pt = URIRef(f"http://www.medsur.org/symptom/{row['PTCode']}")
        g.add((pt, RDF.type, EX.PT))
        g.add((patient, EX.hasPT, pt))
        
    if row["LLTCode"] != np.nan:
        lt = URIRef(f"http://www.medsur.org/symptom/{row['LLTCode']}")
        g.add((lt, RDF.type, EX.LLT))
        g.add((patient, EX.hasLLT, lt))

    if row["ATCode"] != np.nan:
        drug = URIRef(f"http://www.medsur.org/drug/{row['ATCode']}")              
        g.add((drug, RDF.type, EX.Drug))   
        g.add((patient, EX.isGivenDrug, drug))

    # get all side effects for the drug
    df_sideeffects = data_sideeffects.loc[data_sideeffects['ATCode'] == row["ATCode"]]
  
    for index, row2 in df_sideeffects.iterrows():
        
        # check if any side_effect is present in patient file under symptoms (PTCode based)
        if row2["PTCode"] != np.nan and row2["PTCode"] != "nan":
            
            side_effect = URIRef(f"http://www.medsur.org/side_effect/{row2['PTCode']}")
            g.add((side_effect, RDF.type, EX.SideEffects))
            g.add((drug, EX.hasSideEffect, side_effect))
            
            if row2["Frequency"] != np.nan and row2["Frequency"] != "nan":

                g.add((side_effect, EX.hasFrequency, Literal[row2["Frequency"]]))
                
    # check if any side_effect is present in patient file
    if row['is_sideeffect'] == True and row['PTCode'] != np.nan:      
        g.add((patient, EX.suffersSideEffect, Literal("yes")))     
    else:
        g.add((patient, EX.suffersSideEffect, Literal("no")))   

In [ ]:
# run a reasoner
with onto: 
    sync_reasoner(infer_property_values = True)

In [ ]:
# checks for inconsistencies in the ontology
list(default_world.inconsistent_classes())

In [ ]:
# create a networkx graph from the RDF graph
nx_graph = rdflib_to_networkx_digraph(g)

# save ttl file
g.serialize(destination='medsur.ttl', format='turtle')

In [ ]:
# print some stats
print("Number of Nodes: {n}".format(n=nx.number_of_nodes(nx_graph)))
print("Number of Edges: {n}".format(n=nx.number_of_edges(nx_graph)))
print("Density of Graph: {n}".format(n=nx.density(nx_graph)))
print("Clustering coefficient: {n}".format(n=nx.average_clustering(nx_graph)))
print("Degree centrality:", nx.degree_centrality(nx_graph))

In [ ]:
# plot the degree distribution (degree = number of edges)
histdegree = pd.DataFrame(nx.degree_histogram(nx_graph))
degree = dict(nx.degree(nx_graph))

mean_degree = np.mean(list(degree.values()))
mean_degree_centrality = np.mean(list(nx.degree_centrality(nx_graph).values()))

print("Mean degree: {n}".format(n=mean_degree))
print("Mean degree centrality: {n}".format(n=mean_degree_centrality))

fig, ax = plt.subplots(figsize=(12,6)) 
ax.bar(histdegree.index.values,histdegree[0])

plt.title("Mean Degree: {n1}\n Mean Degree Centrality: {n2}".format(n1=mean_degree,n2=mean_degree_centrality))
plt.show()


In [ ]:
# save the triples in a csv file
with open('medsur.csv', 'w') as f: 
    for s, p, o in g:
        f.write(f'{s},{p},{o} \n')